# 向量数据库和嵌入模型

向量数据库

Vector是向量或矢量的意思，向量是数学里的概念，而矢量是物理里的概念，但二者描述的是同一件事。向量的准确定义：向量是用于表示具有大小和方向的量。具体而言，向量可以在不同的维度空间中定义，最常见的是二维和三维空间中的向量，但理论上也可以有更高维的向量。例如，在二维平面上的一个向量可以写作（x,y），这里x和y分别表示该向量沿两个坐标轴方向上的分量；而在三维空间里，则会有一个额外的z坐标，即（x,y,z)。

如我们所见，每个数值向量都有 x 和 y 坐标（或者在多维系统中是 x、y、z，...）。x、y、z... 是这个向量空间的轴，称为维度。对于我们想要表示为向量的一些非数值实体，我们首先需要决定这些维度，并为每个实体在每个维度上分配一个值。

在数学中向量间的相似度测量即相似度计算，有四种常见的计算方法，这里不展开讨论。

* 欧几里得距离 Euclidean distance
* 曼哈顿距离 Manhattan distance
* 点积 Dot product
* 余弦相似度 Cosine similarity

嵌入模型

嵌入模型（Embedding Model）和向量数据库（Vector Database/Vector Store）是一对亲密无间的合作伙伴，也是 AI 技术栈中紧密关联的两大核心组件，两者的协同作用构成了现代语义搜索、推荐系统和 RAG（Retrieval Augmented Generation，检索增强生成）等应用的技术基础。

嵌入（Embedding）的工作原理是将文本、图像和视频转换为称为向量（Vectors）的浮点数数组。这些向量旨在捕捉文本、图像和视频的含义。嵌入数组的长度称为向量的维度（Dimensionality）。

## 1.1 Chroma数据库

* 开源免费：代码公开，社区活跃，易于定制和扩展6。
* 高效存储检索：专为嵌入式向量设计，支持高效存储和检索高维向量，基于近似最近邻算法，如HNSW、IVF、PQ等实现快速检索267。
* 灵活数据模型：除向量数据外，还支持存储元数据和文本内容，检索时可结合向量相似性和元数据过滤条件67。
* 易用性高：提供简单直观API，支持Python和JavaScript等多种编程语言，集成LangChain、LlamaIndex等主流AI框架67。
* 可扩展性强：支持从小规模实验到大规模生产环境的应用，有内存模式、带持久化的内存模式等多种运行模式67。
* 实时更新：支持动态索引更新，可在不中断服务情况下添加、修改或删除向量数据7。


In [73]:
#!pip install chromadb
!pip show chromadb

Name: chromadb
Version: 1.0.13
Summary: Chroma.
Home-page: https://github.com/chroma-core/chroma
Author: 
Author-email: Jeff Huber <jeff@trychroma.com>, Anton Troynikov <anton@trychroma.com>
License: 
Location: d:\programdata\miniconda3\envs\llm310\lib\site-packages
Requires: bcrypt, build, grpcio, httpx, importlib-resources, jsonschema, kubernetes, mmh3, numpy, onnxruntime, opentelemetry-api, opentelemetry-exporter-otlp-proto-grpc, opentelemetry-sdk, orjson, overrides, posthog, pybase64, pydantic, pypika, pyyaml, rich, tenacity, tokenizers, tqdm, typer, typing-extensions, uvicorn
Required-by: 


In [13]:
import chromadb

client = chromadb.PersistentClient(path="./chroma_db")

In [56]:
import chromadb
from chromadb.config import Settings

# settings = Settings(
#     chroma_db_impl="chromadb.db.impl.file.FileDB",
#     persist_directory="./chroma_db")

# client = chromadb.PersistentClient(settings)
# client = chromadb.PersistentClient(settings=settings)
client = chromadb.PersistentClient(path="./chroma_db")

collection_name = "my_db"
# collection = client.get_collection(name=collection_name)

# if collection is not None:
#     print(f"集合 '{collection_name}' 已存在。")
# else:
#     print(f"集合 '{collection_name}' 不存在。")
#     collection = client.create_collection(name="my_db",metadata={"description": "测试集合"})
collection = client.create_collection(name="test3",
                                      metadata={"distance": "cosine"}, #可选的距离函数有 "l2"（欧氏距离）和 "cosine"（余弦相似度）
                                      get_or_create=True)

#添加数据
import numpy as np
documents = ["hello world", "foo bar·"]
metadata = [{"key1": "value1"}, {"key2": "value2"}]
ids = ["id1", "id2"]

vectors = np.random.rand(2,3).tolist()
print(vectors)

collection.add(
    documents=documents,
    metadatas=metadata,
    ids=ids,
    embeddings=vectors
)

query_vectors = np.random.rand(3).tolist()
print(query_vectors)

results = collection.query(
    n_results=2,
    query_embeddings=[query_vectors]
)

print(results)

print("查询结果：")
print("ids:", results["ids"])
print("距离:", results["distances"])
print("文档:", results["documents"])



[[0.2638330112862678, 0.46597636891800753, 0.3552717917062874], [0.44936143585164867, 0.09224615576343798, 0.5011927054055414]]
[0.5663085055936641, 0.47053464316349314, 0.5404820079964674]
{'ids': [['id1', 'id2']], 'embeddings': None, 'documents': [['hello world', 'foo bar·']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'key1': 'value1'}, {'key2': 'value2'}]], 'distances': [[0.18041996657848358, 0.25830531120300293]]}
查询结果：
ids: [['id1', 'id2']]
距离: [[0.18041996657848358, 0.25830531120300293]]
文档: [['hello world', 'foo bar·']]


In [1]:
import pickle

DB_STRING="test/db.string"
DB_INFO="test/db.info"
string_db = []
files_db = []
with open(DB_STRING, 'rb') as cur_file:
    cur_byte_stream = cur_file.read()
    cur_plain_doc = pickle.loads(cur_byte_stream)
    string_db += cur_plain_doc
    #self.string_db.extend(cur_plain_doc)
    print(f" old string db size: {len(string_db)}")

with open(DB_INFO, 'rb') as cur_file:
    cur_byte_stream = cur_file.read()
    cur_plain_doc = pickle.loads(cur_byte_stream)
    files_db += cur_plain_doc
    print(f" old info db size: {len(files_db)}")

print(f"file: {files_db[0]}, string: {string_db[0]}")

 old string db size: 738
 old info db size: 738
file: images/310006490.jpg, string: 图片中是一只大鸟站在水边，背景是树木和水面。鸟的羽毛主要是灰色，腿部有黑色羽毛。背景中还有一只鸟停在水中。


In [2]:
import os
from openai import OpenAI

llm = OpenAI(
    api_key='sk-8a2970d003e849e1a680a377eb16d22b',  # 如果您没有配置环境变量，请在此处用您的API Key进行替换
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"  # 百炼服务的base_url
)

completion = llm.embeddings.create(
    model="text-embedding-v4",
    input='衣服的质量杠杠的，很漂亮，不枉我等了这么久啊，喜欢，以后还来这里买',
    dimensions=1024, # 指定向量维度（仅 text-embedding-v3及 text-embedding-v4支持该参数）
    encoding_format="float"
)

print(completion.model_dump_json())
print(completion.data[0].embedding)

{"data":[{"embedding":[0.00954423751682043,-0.11166470497846603,0.0002610872033983469,-0.04448245093226433,0.018730206415057182,-0.013019428588449955,0.015362495556473732,-0.0032817272003740072,-0.0013390234671533108,0.12496358901262283,0.04709063470363617,-0.03614199161529541,0.027127988636493683,-0.03147018700838089,0.0708509162068367,-0.07320114970207214,-0.038635529577732086,-0.08036649227142334,-0.04929756000638008,-0.03321853280067444,0.023817600682377815,0.05574636906385422,0.028919324278831482,-0.0031061763875186443,-0.027844524011015892,0.03485222905874252,-0.01501855906099081,-0.05649156495928764,-0.008290302008390427,0.06265375763177872,-0.024706102907657623,0.0066888476721942425,-0.042590800672769547,-0.00920030102133751,0.04737725108861923,0.019876662641763687,-0.016265328973531723,0.029979795217514038,0.005911407992243767,0.006860815919935703,0.005273692775517702,-0.019733354449272156,0.0269416905939579,0.0219976045191288,-0.0203352440148592,0.024433819577097893,-0.034938

In [3]:
import chromadb
from chromadb.config import Settings

# settings = Settings(
#     chroma_db_impl="chromadb.db.impl.file.FileDB",
#     persist_directory="./chroma_db")

# client = chromadb.PersistentClient(settings)
# client = chromadb.PersistentClient(settings=settings)
client = chromadb.PersistentClient(path="./chroma_db")

collection_name = "my_db"
# collection = client.get_collection(name=collection_name)

# if collection is not None:
#     print(f"集合 '{collection_name}' 已存在。")
# else:
#     print(f"集合 '{collection_name}' 不存在。")
#     collection = client.create_collection(name="my_db",metadata={"description": "测试集合"})
collection = client.create_collection(name="images",
                                      metadata={"distance": "cosine"}, #可选的距离函数有 "l2"（欧氏距离）和 "cosine"（余弦相似度）
                                      get_or_create=True)

print(f"db group name {collection.name}")


# string_db = []
# files_db = []
cnt = len(string_db)
for i in range(cnt):
    docs = [string_db[i]]
    ids = [files_db[i]]
    completion = llm.embeddings.create(
    model="text-embedding-v4",
    input=string_db[i],
    dimensions=1024, # 指定向量维度（仅 text-embedding-v3及 text-embedding-v4支持该参数）
    encoding_format="float")
    emb = completion.data[0].embedding
    collection.add(documents=docs, ids=ids, embeddings=[emb])
    
print("查询车票")
completion = llm.embeddings.create(
    model="text-embedding-v4",
    input="车票",
    dimensions=1024, # 指定向量维度（仅 text-embedding-v3及 text-embedding-v4支持该参数）
    encoding_format="float")
query_vectors = completion.data[0].embedding
results = collection.query(
    n_results=2,
    query_embeddings=[query_vectors]
)
# #添加数据
# import numpy as np
# documents = ["hello world", "foo bar·"]
# metadata = [{"key1": "value1"}, {"key2": "value2"}]
# ids = ["id1", "id2"]

# vectors = np.random.rand(2,3).tolist()
# print(vectors)

# collection.add(
#     documents=documents,
#     metadatas=metadata,
#     ids=ids,
#     embeddings=vectors
# )

# query_vectors = np.random.rand(3).tolist()
# print(query_vectors)

# results = collection.query(
#     n_results=2,
#     query_embeddings=[query_vectors]
# )

print(results)

print("查询结果：")
print("ids:", results["ids"])
print("距离:", results["distances"])
print("文档:", results["documents"])

db group name images
查询车票
{'ids': [['images/微信图片_20250529155245.png', 'images/微信图片_20250529155234.png']], 'embeddings': None, 'documents': [['这是一张车票，上面有“上海市地铁营业公司”的标志。车票上有“定额车票”字样，车票号码为“3679325”。车票上还有“人民币”字样，以及车票的其他信息。车票的背景是黄色的，上面有一些文字和图案。', '这是一张机票，上面有航班信息、乘客姓名、票价、以及一些附加条款。票号为Z96R097791，航班日期为2016年1月18日，起飞时间为13:55，降落时间为01:02。票价为¥6.5元，乘客名为“海口东站”。']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[None, None]], 'distances': [[0.8257591128349304, 0.8786925673484802]]}
查询结果：
ids: [['images/微信图片_20250529155245.png', 'images/微信图片_20250529155234.png']]
距离: [[0.8257591128349304, 0.8786925673484802]]
文档: [['这是一张车票，上面有“上海市地铁营业公司”的标志。车票上有“定额车票”字样，车票号码为“3679325”。车票上还有“人民币”字样，以及车票的其他信息。车票的背景是黄色的，上面有一些文字和图案。', '这是一张机票，上面有航班信息、乘客姓名、票价、以及一些附加条款。票号为Z96R097791，航班日期为2016年1月18日，起飞时间为13:55，降落时间为01:02。票价为¥6.5元，乘客名为“海口东站”。']]


In [4]:
text = "夜晚的街景"
completion = llm.embeddings.create(
    model="text-embedding-v4",
    input=text,
    dimensions=1024, # 指定向量维度（仅 text-embedding-v3及 text-embedding-v4支持该参数）
    encoding_format="float")
query_vectors = completion.data[0].embedding
results = collection.query(
    n_results=3,
    query_embeddings=[query_vectors]
)

print(results)

for item in results["ids"][0]:
    print("ids:", item)

for item in results["distances"][0]:
	print("距离:", item)

for item in results["documents"][0]:
    print("文档:", item)


{'ids': [['images/{no.30001} (23).jpeg', 'images/100003040 (1).jpeg', 'images/10219.jpeg']], 'embeddings': None, 'documents': [['这是一张夜晚的街景照片。街道两旁是欧式风格的建筑，街道上铺着石板。街道上悬挂着串灯，灯光温暖。街道上停放着几辆汽车，街道中央有一个人坐在长椅上。', '这张图片展示了一个夜晚的街景，街道上有车辆行驶。建筑物上装饰着霓虹灯和招牌，招牌上有中文和英文。街道两旁有商店和餐馆，整体氛围热闹且充满都市气息。', '这张图片展示了一个夜晚的街景，街道上有车辆行驶。建筑物上装饰着霓虹灯和招牌，招牌上有中文和英文。街道两旁有商店和餐馆，整体氛围热闹且充满都市气息。']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[None, None, None]], 'distances': [[0.5939126014709473, 0.6031597852706909, 0.6031597852706909]]}
ids: images/{no.30001} (23).jpeg
ids: images/100003040 (1).jpeg
ids: images/10219.jpeg
距离: 0.5939126014709473
距离: 0.6031597852706909
距离: 0.6031597852706909
文档: 这是一张夜晚的街景照片。街道两旁是欧式风格的建筑，街道上铺着石板。街道上悬挂着串灯，灯光温暖。街道上停放着几辆汽车，街道中央有一个人坐在长椅上。
文档: 这张图片展示了一个夜晚的街景，街道上有车辆行驶。建筑物上装饰着霓虹灯和招牌，招牌上有中文和英文。街道两旁有商店和餐馆，整体氛围热闹且充满都市气息。
文档: 这张图片展示了一个夜晚的街景，街道上有车辆行驶。建筑物上装饰着霓虹灯和招牌，招牌上有中文和英文。街道两旁有商店和餐馆，整体氛围热闹且充满都市气息。


In [ ]:
print(collection.name)